In [12]:
from __future__ import print_function, division
import os
import pandas as pd
import time
from pydub import AudioSegment

In [2]:
DATASET = "../datasets/matches.csv"
DOWNLOADS_DIR = "../downloads/"
OUTPUT_DIR = "../transforms/"

In [3]:
if (not os.path.exists(DOWNLOADS_DIR)):
    os.mkdir(DOWNLOADS_DIR)
    
if (not os.path.exists(OUTPUT_DIR)):
    os.mkdir(OUTPUT_DIR)

In [4]:
df = pd.read_csv(DATASET, header=0)
df.head()

,salami_id,salami_length,youtube_id,youtube_length,coverage,coverage_percent,onset_in_youtube,onset_in_salami
0,2,264.800000,DKzFw_i6Ffg,265.0,261.4,0.987160,0.6,0.5
1,3,276.662812,E6iGez4BLjQ,277.0,270.3,0.977002,0.3,0.5
2,4,236.094667,ct8gCRPST9s,235.0,232.5,0.984774,0.4,0.5
3,5,256.940417,IBdrzoz9o0o,258.0,248.1,0.965594,6.0,6.0
4,6,170.466667,QurVLnYoFqo,174.0,163.5,0.959132,4.8,0.2


In [17]:
for _,item in df.head().iterrows():
    filename = str(item.salami_id) + ".m4a"
    input_file = DOWNLOADS_DIR + filename
    output_file = OUTPUT_DIR + filename
    if (os.path.exists(input_file)):
        input_file = AudioSegment.from_file(input_file, "mp3")
        yt_start = item["onset_in_youtube"] - item["onset_in_salami"]
        yt_end = yt_start + item["salami_length"]
        yt_length = item["youtube_length"]
        if yt_end > yt_length:
            pad = AudioSegment.silent(duration=(yt_end - yt_length)*1000)
            out = input_file + pad
        if yt_start < 0:
            pad = AudioSegment.silent(duration = (yt_start*-1)*1000)
            out = pad + input_file
            yt_start = 0
        out = out[yt_start:(yt_start + item["salami_length"])*1000]
        out.export(output_file, format="mp3")